<a href="https://colab.research.google.com/github/emilysoueu/DataExtractionTheSpaceDevs/blob/main/API_TheSpaceDevs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração de Dados via API

In [1]:
#Bibliotecas
import pandas as pd
import requests
from datetime import datetime

In [11]:
#Lauch Endpoint
launch_base_url = 'https://lldev.thespacedevs.com/2.2.0/launch/upcoming/'

# Teste filtering past launches

In [4]:
# Time frame: now - 31 days ago
from datetime import datetime, timedelta # Import timedelta
now = datetime.now()
month_ago = now - timedelta(days=31)

# Adding the time frame to the filters
net_filters = f'net__gte={month_ago.isoformat()}&net__lte={now.isoformat()}'

Setting response mode

In [5]:
 # Set mode to detailed to include all related objects
 mode = 'mode=detailed'

Limiting

In [7]:
# Limit returned results to just 2 per query
limit = 'limit=2'


Ordering

In [8]:
 # Ordering the results by ascending T-0 (NET)
 ordering = 'ordering=net'

Assembly query URL

In [20]:
# Assemble the query URL
launch_base_url = "https://api.spacexdata.com/v4/launches/query" # Add the base URL here
# Define the filters you want to use in your query.
# For example, to filter by launches with a 'success' landing attempt:
filters = "filter=landing_attempt:success"
# Define how you want to combine the filters (e.g., 'and', 'or')
mode = 'mode=and'
query_url = launch_base_url + '?' + '&'.join(
    (filters, mode, limit, ordering)
)
print(f'query URL: {query_url}')

query URL: https://api.spacexdata.com/v4/launches/query?filter=landing_attempt:success&mode=and&limit=2&ordering=net


Pagination through all the results

In [26]:
 # Paginating through all the results
next_url = results['next']
 while next_url:
     # Requesting next data
     next_results = get_results(next_url)

     # Checking if it was succesful
     if not results:
         # Quitting the program as an example
         # use your own error handling here
         quit()

           # Printing next results
     print(next_results)

     # Adding to the original results dictionary
     results['results'] += next_results['results']

     # Updating the next URL
     next_url = next_results['next']

KeyError: 'next'

# Outro Tests

In [39]:
#Bibliotecas
import pandas as pd
import requests
from datetime import datetime


url = 'https://lldev.thespacedevs.com/2.2.0/launch/upcoming/'

response = requests.request('GET', url)

In [36]:
print (response)


<Response [200]>


In [37]:
response.json()

{'count': 325,
 'next': 'https://lldev.thespacedevs.com/2.2.0/launch/upcoming/?limit=10&offset=10',
 'previous': None,
 'results': [{'id': 'e8c62aa0-6bef-48ac-8ef6-b8589479c6be',
   'url': 'https://lldev.thespacedevs.com/2.2.0/launch/e8c62aa0-6bef-48ac-8ef6-b8589479c6be/',
   'slug': 'falcon-9-block-5-nrol-113',
   'name': 'Falcon 9 Block 5 | NROL-113',
   'status': {'id': 8,
    'name': 'To Be Confirmed',
    'abbrev': 'TBC',
    'description': 'Awaiting official confirmation - current date is known with some certainty.'},
   'last_updated': '2024-09-01T01:57:51Z',
   'net': '2024-09-05T01:28:00Z',
   'window_end': '2024-09-05T05:58:00Z',
   'window_start': '2024-09-05T01:28:00Z',
   'net_precision': {'id': 2,
    'name': 'Hour',
    'abbrev': 'HR',
    'description': 'The T-0 is accurate to the hour.'},
   'probability': None,
   'weather_concerns': None,
   'holdreason': '',
   'failreason': '',
   'hashtag': None,
   'launch_service_provider': {'id': 121,
    'url': 'https://lldev.

In [40]:
#/2.2.0/api-throttle/
#Para monitorar a quantidade de requisições

url = 'https://lldev.thespacedevs.com//2.2.0/api-throttle/'

response = requests.request('GET', url)

print (response)

response.json()

<Response [200]>


{'your_request_limit': 15,
 'limit_frequency_secs': 3600,
 'current_use': 0,
 'next_use_secs': 0,
 'ident': '35.231.16.229'}

# Transformando os dados da API em um Dataset

In [41]:
import pandas as pd

# Exemplo de dados JSON
data = {
    'count': 325,
    'next': 'https://lldev.thespacedevs.com/2.2.0/launch/upcoming/?limit=10&offset=10',
    'previous': None,
    'results': [
        {
            'id': 'e8c62aa0-6bef-48ac-8ef6-b8589479c6be',
            'name': 'Falcon 9 Block 5 | NROL-113',
            'status': {'name': 'To Be Confirmed'},
            'launch_service_provider': {'name': 'SpaceX'},
            'rocket': {'configuration': {'name': 'Falcon 9'}},
            'mission': {'name': 'NROL-113', 'description': 'Third batch of satellites for a reconnaissance satellite constellation'},
            'net': '2024-09-05T01:28:00Z'
        },
        # Adicionar outros lançamentos aqui
    ]
}

# Extraindo as informações e transformando em DataFrame
launches = data['results']
df = pd.json_normalize(launches)

# Exibir o DataFrame para ver a estrutura
print(df)

# Salvando como uma planilha Excel
df.to_excel('launches.xlsx', index=False)


                                     id                         name  \
0  e8c62aa0-6bef-48ac-8ef6-b8589479c6be  Falcon 9 Block 5 | NROL-113   

                    net      status.name launch_service_provider.name  \
0  2024-09-05T01:28:00Z  To Be Confirmed                       SpaceX   

  rocket.configuration.name mission.name  \
0                  Falcon 9     NROL-113   

                                 mission.description  
0  Third batch of satellites for a reconnaissance...  
